In [1]:
import pandas as pd
import numpy as np
import ast
import math

/Users/zhuhongyun/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/zhuhongyun/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
pip install pandas

  Using cached numpy-1.26.4-cp39-cp39-macosx_10_9_x86_64.whl (20.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
scipy 1.9.1 requires numpy<1.25.0,>=1.18.5, but you have numpy 1.26.4 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.26.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
file_path = 'UoB_Set01_2025-05-16LOBs.txt'

In [3]:
# Parsing single-line text
def parse_line_corrected(line):
    line = line.replace('Exch0', "'Exch0'")
    data = ast.literal_eval(line)
    return {
        'timestamp': data[0],
        'exchange': data[1],
        'bids': data[2][0][1],
        'asks': data[2][1][1]
    }

In [4]:
# Parses incoming text line by line
parsed_data_corrected = []
with open(file_path, 'r') as file:
    for line in file:
        parsed_data_corrected.append(parse_line_corrected(line))

In [5]:
df_corrected = pd.DataFrame(parsed_data_corrected)

In [6]:
df_corrected = df_corrected[['exchange', 'timestamp', 'bids', 'asks']]
df_corrected.head()

,exchange,timestamp,bids,asks
0,Exch0,0.000,[],[]
1,Exch0,2.294,[],"[[158, 5]]"
2,Exch0,2.604,"[[1, 9]]","[[158, 5]]"
3,Exch0,3.317,"[[2, 9]]","[[158, 5]]"
4,Exch0,3.968,"[[3, 4], [2, 9]]","[[158, 5]]"


In [7]:
# Sort bids and asks and take the top two
def process_row(row):
    bids_sorted = sorted(row['bids'], key=lambda x: x[0], reverse=True)[:2] if row['bids'] else []
    asks_sorted = sorted(row['asks'], key=lambda x: x[0])[:2] if row['asks'] else []
    return bids_sorted, asks_sorted

In [8]:
df_corrected[['bids', 'asks']] = df_corrected.apply(lambda row: process_row(row), axis=1, result_type='expand')
df_filtered = (df_corrected[(df_corrected['bids'].apply(len) >= 2) & (df_corrected['asks'].apply(len) >= 2)]).reset_index(drop=True)
df_filtered.head()

,exchange,timestamp,bids,asks
0,Exch0,4.309,"[[4, 4], [2, 9]]","[[158, 5], [532, 1]]"
1,Exch0,4.433,"[[5, 9], [4, 4]]","[[158, 5], [532, 1]]"
2,Exch0,4.557,"[[5, 9], [4, 4]]","[[157, 3], [158, 5]]"
3,Exch0,4.743,"[[142, 1], [5, 9]]","[[157, 3], [158, 5]]"
4,Exch0,4.805,"[[142, 1], [5, 9]]","[[157, 3], [158, 5]]"


In [9]:
def remove_max_bid_outliers(bids, mean_price, std_price, threshold=3):
    if not bids:
        return True  
    # Calculate z-score for the maximum bid price
    max_bid_price = max(bid[0] for bid in bids)
    z_score = abs(max_bid_price - mean_price) / std_price
    
    # Return True if the z-score is within the threshold, indicating it's not an outlier
    return z_score <= threshold

# Calculate the mean and standard deviation for the maximum bid prices
max_bid_prices = [max(bid[0] for bid in bids_list) for bids_list in df_filtered['bids'] if bids_list]
mean_max_price = np.mean(max_bid_prices)
std_max_price = np.std(max_bid_prices)

# Filter the DataFrame rows
df_filtered_no_max_outliers = df_filtered[df_filtered['bids'].apply(
    lambda bids: remove_max_bid_outliers(bids, mean_max_price, std_max_price, 3)
)]

# Reset the index after the filter operation
df_filtered_no_max_outliers = df_filtered_no_max_outliers.reset_index(drop=True)

df_filtered_no_max_outliers

,exchange,timestamp,bids,asks
0,Exch0,4.743,"[[142, 1], [5, 9]]","[[157, 3], [158, 5]]"
1,Exch0,4.805,"[[142, 1], [5, 9]]","[[157, 3], [158, 5]]"
2,Exch0,4.836,"[[142, 1], [5, 9]]","[[156, 3], [158, 5]]"
3,Exch0,5.456,"[[127, 1], [81, 1]]","[[135, 1], [155, 3]]"
4,Exch0,5.487,"[[127, 1], [81, 1]]","[[134, 3], [135, 1]]"
...,...,...,...,...
317338,Exch0,30599.542,"[[132, 5], [131, 2]]","[[141, 1], [142, 1]]"
317339,Exch0,30599.790,"[[132, 5], [131, 2]]","[[141, 1], [142, 1]]"
317340,Exch0,30599.821,"[[132, 5], [131, 2]]","[[141, 1], [142, 1]]"
317341,Exch0,30599.945,"[[132, 5], [131, 2]]","[[141, 1], [444, 5]]"


In [10]:
def remove_min_bid_outliers(bids, mean_min_price, std_min_price, threshold=3):
    if not bids:
        return True
    min_bid_price = min(bid[0] for bid in bids)
    z_score = (min_bid_price - mean_min_price) / std_min_price
    return abs(z_score) <= threshold

# Calculate mean and standard deviation for the minimum bid prices across the DataFrame
min_bid_prices = [min(bid[0] for bid in bids_list) for bids_list in df_filtered_no_max_outliers['bids'] if bids_list]
mean_min_price = np.mean(min_bid_prices)
std_min_price = np.std(min_bid_prices)

# Apply the outlier removal function to each row
df_filtered_no_min_outliers = df_filtered_no_max_outliers[df_filtered_no_max_outliers['bids'].apply(
    lambda bids: remove_min_bid_outliers(bids, mean_min_price, std_min_price, 3)
)]

# Reset index
df_filtered_no_min_outliers.reset_index(drop=True, inplace=True)

# Display the first few rows
df_filtered_no_min_outliers

,exchange,timestamp,bids,asks
0,Exch0,5.611,"[[128, 4], [127, 1]]","[[134, 3], [135, 1]]"
1,Exch0,6.851,"[[129, 4], [128, 1]]","[[130, 3], [132, 1]]"
2,Exch0,6.882,"[[129, 4], [128, 1]]","[[130, 3], [132, 1]]"
3,Exch0,6.944,"[[129, 1], [128, 1]]","[[132, 1], [134, 3]]"
4,Exch0,7.161,"[[129, 1], [128, 1]]","[[132, 5], [134, 3]]"
...,...,...,...,...
308497,Exch0,30599.542,"[[132, 5], [131, 2]]","[[141, 1], [142, 1]]"
308498,Exch0,30599.790,"[[132, 5], [131, 2]]","[[141, 1], [142, 1]]"
308499,Exch0,30599.821,"[[132, 5], [131, 2]]","[[141, 1], [142, 1]]"
308500,Exch0,30599.945,"[[132, 5], [131, 2]]","[[141, 1], [444, 5]]"


In [11]:
def remove_min_ask_outliers(asks, mean_price, std_price, threshold=3):
    if not asks:
        return True  # Keep rows with no asks as they're not outliers
    min_ask_price = min(ask[0] for ask in asks)
    z_score = (min_ask_price - mean_price) / std_price
    return abs(z_score) <= threshold

# Calculate mean and standard deviation for minimum ask prices
min_ask_prices = [min(ask[0] for ask in asks_list) for asks_list in df_filtered['asks'] if asks_list]
mean_min_ask_price = np.mean(min_ask_prices)
std_min_ask_price = np.std(min_ask_prices)

# Apply the function to filter rows based on minimum ask price outliers
df_filtered_no_min_ask_outliers = df_filtered_no_min_outliers[df_filtered_no_min_outliers['asks'].apply(
    lambda asks: remove_min_ask_outliers(asks, mean_min_ask_price, std_min_ask_price, 3)
)]

# Reset the index after filtering
df_filtered_no_min_ask_outliers.reset_index(drop=True, inplace=True)

df_filtered_no_min_ask_outliers

,exchange,timestamp,bids,asks
0,Exch0,5.611,"[[128, 4], [127, 1]]","[[134, 3], [135, 1]]"
1,Exch0,6.851,"[[129, 4], [128, 1]]","[[130, 3], [132, 1]]"
2,Exch0,6.882,"[[129, 4], [128, 1]]","[[130, 3], [132, 1]]"
3,Exch0,6.944,"[[129, 1], [128, 1]]","[[132, 1], [134, 3]]"
4,Exch0,7.161,"[[129, 1], [128, 1]]","[[132, 5], [134, 3]]"
...,...,...,...,...
300088,Exch0,30599.542,"[[132, 5], [131, 2]]","[[141, 1], [142, 1]]"
300089,Exch0,30599.790,"[[132, 5], [131, 2]]","[[141, 1], [142, 1]]"
300090,Exch0,30599.821,"[[132, 5], [131, 2]]","[[141, 1], [142, 1]]"
300091,Exch0,30599.945,"[[132, 5], [131, 2]]","[[141, 1], [444, 5]]"


In [12]:
def remove_max_ask_outlier(asks, mean_price, std_price, threshold=3):
    if not asks:
        return True
    max_ask_price = max(ask[0] for ask in asks)
    z_score = (max_ask_price - mean_price) / std_price
    return abs(z_score) <= threshold

# Calculate mean and standard deviation for maximum ask prices
max_ask_prices = [max(ask[0] for ask in asks_list) for asks_list in df_filtered_no_min_ask_outliers['asks'] if asks_list]
mean_max_ask_price = np.mean(max_ask_prices)
std_max_ask_price = np.std(max_ask_prices)

# Apply the function to filter rows based on maximum ask price outliers
df_filtered_no_max_ask_outliers = df_filtered_no_min_ask_outliers[df_filtered_no_min_ask_outliers['asks'].apply(
    lambda asks: remove_max_ask_outlier(asks, mean_max_ask_price, std_max_ask_price, 3)
)]

# Reset the index after filtering
df_filtered_no_max_ask_outliers.reset_index(drop=True, inplace=True)

df_filtered_no_max_ask_outliers

,exchange,timestamp,bids,asks
0,Exch0,5.611,"[[128, 4], [127, 1]]","[[134, 3], [135, 1]]"
1,Exch0,6.851,"[[129, 4], [128, 1]]","[[130, 3], [132, 1]]"
2,Exch0,6.882,"[[129, 4], [128, 1]]","[[130, 3], [132, 1]]"
3,Exch0,6.944,"[[129, 1], [128, 1]]","[[132, 1], [134, 3]]"
4,Exch0,7.161,"[[129, 1], [128, 1]]","[[132, 5], [134, 3]]"
...,...,...,...,...
294447,Exch0,30599.542,"[[132, 5], [131, 2]]","[[141, 1], [142, 1]]"
294448,Exch0,30599.790,"[[132, 5], [131, 2]]","[[141, 1], [142, 1]]"
294449,Exch0,30599.821,"[[132, 5], [131, 2]]","[[141, 1], [142, 1]]"
294450,Exch0,30599.945,"[[132, 5], [131, 2]]","[[141, 1], [444, 5]]"


In [13]:
def split_bids_asks(row):
    # Splitting bids into prices and sizes
    bid_price1, bid_size1 = row['bids'][0]
    bid_price2, bid_size2 = row['bids'][1]
    ask_price1, ask_size1 = row['asks'][0]
    ask_price2, ask_size2 = row['asks'][1]  
    return pd.Series([bid_price1, bid_price2, bid_size1, bid_size2, ask_price1, ask_price2, ask_size1, ask_size2])

# Apply the function to each row of the DataFrame and create new columns
df_final_filtered = df_filtered_no_max_ask_outliers.copy()
df_final_filtered[['bid_price1', 'bid_price2', 'bid_size1', 'bid_size2', 'ask_price1', 'ask_price2', 'ask_size1', 'ask_size2']] = df_final_filtered.apply(split_bids_asks, axis=1)
df_final_filtered.drop(['bids', 'asks'], axis=1, inplace=True)

# Display the first few rows of the DataFrame to verify the changes
df_final_filtered.drop(columns=['exchange'], inplace=True)
df_final_filtered.head(10)

,timestamp,bid_price1,bid_price2,bid_size1,bid_size2,ask_price1,ask_price2,ask_size1,ask_size2
0,5.611,128,127,4,1,134,135,3,1
1,6.851,129,128,4,1,130,132,3,1
2,6.882,129,128,4,1,130,132,3,1
3,6.944,129,128,1,1,132,134,1,3
4,7.161,129,128,1,1,132,134,5,3
5,7.657,129,128,1,1,132,134,5,3
6,8.122,129,128,1,1,132,134,5,3
7,8.246,129,128,1,1,131,132,1,4
8,8.463,129,128,1,1,131,132,1,4
9,8.649,129,128,1,1,131,132,1,4


In [14]:
# Feature extraction
# Function to calculate first WAP
def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

# Function to calculate second WAP
def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

In [15]:
#Function used to calculate log return
def log_return(series):
    # 确保Series中没有空值，并且所有输入都转换为浮点数
    series = series.dropna().astype(float)
    return np.log(series).diff()


# Calculate the realized volatility
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

In [16]:
def Feature_extraction(df):
    data_dict = {}
    # Calculate Wap
    data_dict['wap1'] = calc_wap1(df)
    data_dict['wap2'] = calc_wap2(df)
    data_dict['log_return1'] = np.log(data_dict['wap1']) - np.log(data_dict['wap1'].shift(1))
    data_dict['log_return2'] = np.log(data_dict['wap2']) - np.log(data_dict['wap2'].shift(1))
    # Calculate wap balance
    data_dict['wap_balance'] = abs(data_dict['wap1'] - data_dict['wap2'])
    # Calculate spread
    data_dict['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    data_dict['price_spread2'] = (df['ask_price2'] - df['bid_price2']) / ((df['ask_price2'] + df['bid_price2']) / 2)
    data_dict['bid_spread'] = df['bid_price1'] - df['bid_price2']
    data_dict['ask_spread'] = df['ask_price1'] - df['ask_price2']
    #Calculate the spread relative to the average price
    data_dict["bid_ask_spread"] = abs(data_dict['bid_spread'] - data_dict['ask_spread'])
    data_dict['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    #To quantify the current pressure imbalance between buyers and sellers
    data_dict['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))

     # Dict for aggregations
    create_feature_dict = {
        'wap1': [np.sum, np.mean, np.std],
        'wap2': [np.sum, np.mean, np.std],
        'log_return1': [np.sum, realized_volatility, np.mean, np.std],
        'log_return2': [np.sum, realized_volatility, np.mean, np.std],
        'wap_balance': [np.sum, np.mean, np.std],
        'price_spread':[np.sum, np.mean, np.std],
        'price_spread2':[np.sum, np.mean, np.std],
        'bid_spread':[np.sum, np.mean, np.std],
        'ask_spread':[np.sum, np.mean, np.std],
        'total_volume':[np.sum, np.mean, np.std],
        'volume_imbalance':[np.sum, np.mean, np.std],
        "bid_ask_spread":[np.sum, np.mean, np.std],
    }
    
    
    name_dict = {
            np.sum:"sum",
            np.mean:"mean",
            np.std:"std",
            realized_volatility:"realized_volatility"
    }
    


#     # Define window size and step size
#     window_size = 100  # Window size in seconds
#     step_size = 10     # Window step size in seconds

#     # Create a new column 'window_id' to represent the window to which each timestamp belongs
#     df['window_id'] = df['timestamp'].apply(lambda x: x // step_size)

#     # Grouping and aggregation calculations.
#     aggregated_df = df.groupby('window_id').agg(create_feature_dict)

#     # The window ID can be used to calculate the start and end timestamps of each window
#     aggregated_df['window_start'] = aggregated_df.index * step_size
#     aggregated_df['window_end'] = aggregated_df['window_start'] + window_size
    
#     # Reset index so window ID is a column
#     aggregated_df.reset_index(drop=True, inplace=True)

#     df['timestamp'] = pd.to_datetime(df['timestamp']) # 我把这行注释掉了


 # print(data_dict)
    df_t_1 = {}
    for key,value in create_feature_dict.items():
        for func in value:
            df_t_1[key+"_"+name_dict[func]] = func(data_dict[key])

    # print(df_new.index)
    return pd.DataFrame([df_t_1])





In [36]:
def sequentially(df_base):
    lines = []
    max_ts = math.floor(df_base['timestamp'].max())-600
    interval = 600
    interval_step = [100,200,300,400,500]
    for j in range(0,max_ts,10):
        df_t = df_base[(df_base['timestamp'] >= j) & (df_base['timestamp'] < j+interval)]
        # print(df_t)
        df_total = Feature_extraction(df_t.copy())
        # print(df_total)
        df_total['range'] = "{}-{}".format(j,j+600)
            # # print(df_sum)
        lines.append(df_total)
        for step in interval_step:
            df_step = df_t[(df_t['timestamp'] >= j) & (df_t['timestamp'] < j+step)]

            df_step_sum = Feature_extraction(df_step.copy())
            df_step_sum['range'] = "{}-{}".format(j,j+step)
            lines.append(df_step_sum)

    result = pd.concat(lines,axis=0)
    
    return result


In [37]:
sequentially(df_final_filtered).to_csv("result.csv",index=False)

In [38]:
df=pd.read_csv("result.csv")

# Set loop value
cycle_values = [600, 100, 200, 300, 400, 500]

# Generate a cyclic sequence whose length is the same as the number of rows of df
num_rows = len(df)
cycle_sequence = cycle_values * (num_rows // len(cycle_values)) + cycle_values[:num_rows % len(cycle_values)]

# Assign loop sequence to 'range' column
df['range'] = cycle_sequence

# Add an index to each range loop
df['index'] = (df.index // 6) + 1

#Merge rows with the same index into one row
pivot_df = df.pivot(index='index', columns='range')

num_rows = len(pivot_df)

# Create a looping timeline that increments every 10
start_time = [(i * 10)  for i in range(num_rows)]
end_time = [600 + i * 10 for i in range(num_rows)]

# Add timeline to DataFrame
pivot_df['start_time'] = start_time
pivot_df['end_time'] = end_time


pivot_df.head(15)

wap1_sum                                               \
range            100            200            300            400   
index                                                               
1      145489.444864  304610.494382  458304.778378  614536.818320   
2      161644.518357  316451.977788  472597.385809  629864.111689   
3      168875.936672  325101.643156  478064.699807  634869.778240   
4      170468.306495  325216.702741  476562.021095  638849.413888   
5      170080.243431  329480.287079  484123.454684  638909.266738   
6      174333.787570  336740.658110  491963.908247  649834.839989   
7      175105.407014  332212.591108  488551.926501  649175.385044   
8      175844.719682  327858.942920  485598.007606  648602.168530   
9      169998.175565  322360.174753  481534.501872  650567.253453   
10     164571.990355  317403.919763  476228.229317  644256.322701   
11     159121.049519  312815.333514  469047.373456  638367.900478   
12     154807.459431  310952.867452  468219.593332  633990.900398   
13     156225.706484  309188.763134  465993.841567  633968.129413   
14     154748.396245  306093.714600  468381.107392  632531.027844   
15     159400.043648  314043.211253  468829.023307  635208.858064   

                                      wap1_mean                          \
range            500            600         100         200         300   
index                                                                     
1      783857.345342  926625.378079  158.831272  161.511397  158.090644   
2      795635.418755  939124.890795  159.885775  160.880517  159.553473   
3      802844.066086  940691.008905  160.376008  164.525123  160.478248   
4      802999.334340  945939.483292  159.465207  164.666685  161.546448   
5      805289.101495  955236.932529  164.011807  164.657815  162.785291   
6      809963.618161  958787.699989  161.719655  162.598097  161.617578   
7      807061.145939  956001.705079  162.435442  161.033733  160.707871   
8      801919.267329  950801.587949  165.734891  161.030915  161.973985   
9      798344.688946  948469.355011  166.501641  160.858371  162.078257   
10     790912.441471  942925.458470  165.232922  159.179498  161.817271   
11     781135.933215  933546.246565  164.042319  157.748529  161.517691   
12     777480.372438  930737.227834  161.932489  159.381275  161.010864   
13     771815.072233  931970.556994  169.258620  160.534145  161.411099   
14     775471.176796  933056.037629  170.803969  162.729248  162.858521   
15     785156.689098  939573.383163  165.352742  162.128658  162.056351   

                   ... bid_ask_spread_mean            bid_ask_spread_std  \
range         400  ...                 500        600                100   
index              ...                                                     
1      160.873513  ...           76.594332  77.449508          98.517849   
2      160.720620  ...           76.509050  76.933924          99.026083   
3      161.134461  ...           78.697166  77.607860          99.780550   
4      161.939015  ...           77.249899  75.162107          91.455613   
5      162.572333  ...           77.748290  73.679919          91.049007   
6      162.865875  ...           77.861980  72.960315          87.601298   
7      162.172217  ...           75.818236  72.074248          82.489126   
8      162.965369  ...           76.524354  73.583277          86.721658   
9      163.049437  ...           76.408853  72.737741          92.415255   
10     163.309588  ...           76.571602  74.280122          97.023174   
11     162.932083  ...           79.084478  74.925976          98.121021   
12     162.312058  ...           77.601934  75.436064         103.733400   
13     163.099596  ...           78.418031  76.262056         105.764247   
14     163.149607  ...           77.567899  75.483716         103.453693   
15     161.507465  ...           75.362931  76.084717         100.948782   

                                                 

In [39]:
pivot_df.to_csv("pivot_result.csv",index=False)